In [ ]:
from ipyradiant import (
    CustomURIRef,
    FileManager,
    MultiPanelSelect,
    ObjectLiteralApp,
    PathLoader,
)
from rdflib import URIRef

## Start by using the `ObjectLiteralApp`

### We also utilize the `CustomURIRef` class from `ipyradiant`. This allows us to get a 'pretty' representation of a URIRef while mainting access to the valuable URIRef.

We can borrow the loading widgets seen in other examples like the Tab App in order to
get a graph to play around with.

In [ ]:
lw = FileManager(loader=PathLoader(path="data"))
lw

Now we can use the `ObjectLiteralApp` to move predicates around based on what we which
ones we want to select. The 'Add predicates where object is a literal' button will
automatically move all predicates to the 'predicates to collapse' side if that predicate
is _always_ associated with a literal object.

In [ ]:
graph = lw.graph
ola = ObjectLiteralApp(graph=graph, namespaces={URIRef("http://www.w3.org/"): "w3"})
ola

Now lets say we want to access one of the predicates. We can do this while retaining
full access to the URI as follows:

In [ ]:
first_pred = ola.multiselect.selected_things_list[0]
first_pred.uri